In [1]:
import sqlite3
import pandas as pd


## Connection à la BDD

In [2]:
connection = sqlite3.connect('data/blog.sqlite')
cursor = connection.cursor()


# Analyse Données:

## Schéma BDD

<img src="schema.png">

## Liste Tables

In [27]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
df_tables = pd.read_sql_query(query, connection)
df_tables


,name
0,sqlite_sequence
1,users
2,posts
3,likes


## 1. Table users

In [78]:
query = "PRAGMA table_info(users)"
df_users_col = pd.read_sql_query(query, connection, index_col='cid')
df_users_col


,name,type,notnull,dflt_value,pk
cid,,,,,
0,id,INTEGER,0,None,1
1,first_name,VARCHAR(20),0,None,0
2,last_name,VARCHAR(20),0,None,0


In [77]:
query = "SELECT * FROM users LIMIT 10"
df_users_rows = pd.read_sql_query(query, connection, index_col='id')
df_users_rows


,first_name,last_name
id,,
1,Cory,Bush
2,Craig,Good
3,Todd,Smith
4,Ashlee,Lee
5,Kyle,Johnson
6,Justin,Tapia
7,Karen,Brown
8,Michael,Pugh
9,Maria,Mccarty


## 2. Table posts

In [76]:
query = "PRAGMA table_info(posts)"
df_posts_col = pd.read_sql_query(query, connection, index_col='cid')
df_posts_col


,name,type,notnull,dflt_value,pk
cid,,,,,
0,id,INTEGER,0,None,1
1,title,TEXT,0,None,0
2,content,TEXT,0,None,0
3,user_id,INTEGER,0,None,0


In [75]:
query = "SELECT * FROM posts LIMIT 10"
df_posts_rows = pd.read_sql_query(query, connection, index_col='id')
df_posts_rows


,title,content,user_id
id,,,
1,Feel help view real color.,Store theory hotel bar move. Blue summer leg a...,50
2,Help give in real instead.,Happy manager worry meet song. Economic hotel ...,50
3,Remain get structure huge never worry could.,Imagine late offer admit. Must scientist hot s...,50
4,Career ahead line look.,Send face nothing house realize. Team son trip...,50
5,Develop manage similar case majority require e...,Either seat way effect story head name artist....,50
6,Than billion magazine strategy.,Pass top several improve. She summer governmen...,51
7,Above matter trip debate trial.,Even employee bar school back. According inter...,51
8,Kid system different question.,Fall support although experience maybe technol...,51
9,Deep scene natural rate also.,Way factor give baby government. Order report ...,51


## 3. Table likes

In [74]:
query = "PRAGMA table_info(likes)"
df_likes_col = pd.read_sql_query(query, connection, index_col='cid')
df_likes_col


,name,type,notnull,dflt_value,pk
cid,,,,,
0,id,INTEGER,0,None,1
1,created_at,TEXT,0,None,0
2,user_id,INTEGER,0,None,0
3,post_id,INTEGER,0,None,0


In [73]:
query = "SELECT * FROM likes LIMIT 10"
df_likes_rows = pd.read_sql_query(query, connection, index_col='id')
df_likes_rows


,created_at,user_id,post_id
id,,,
1,2019-01-17,24,54
2,2019-12-28,12,136
3,2019-06-06,40,84
4,2019-05-04,21,53
5,2019-11-11,16,89
6,2019-02-02,43,26
7,2019-02-06,4,113
8,2019-07-05,35,6
9,2019-08-07,33,5


# Exercice:

---
## Q1. Retourner le top 3 des articles les plus liké.
---

In [72]:
query = """
           SELECT posts.id, posts.title, COUNT(likes.post_id) AS nombre_de_likes
           FROM posts
           JOIN likes ON posts.id = likes.post_id
           GROUP BY posts.id, posts.title
           ORDER BY nombre_de_likes DESC
           LIMIT 3;
"""
df_top3_articles = pd.read_sql_query(query, connection, index_col='id')
df_top3_articles


,title,nombre_de_likes
id,,
143,Half imagine another.,84
83,Side foot leader popular.,82
99,Area paper whatever mean.,81


---
## Q2. Trouvez les trois utilisateurs qui ont le plus liké
---

In [71]:
query = """
        SELECT users.id, users.first_name, users.last_name, COUNT(likes.user_id) AS total_likes
        FROM users
        JOIN likes ON users.id = likes.user_id
        GROUP BY users.id, users.first_name, users.last_name
        ORDER BY total_likes DESC
        LIMIT 3
"""
df_top3_users = pd.read_sql_query(query, connection, index_col='id')
df_top3_users


,first_name,last_name,total_likes
id,,,
43,Michael,Allen,236
12,Donna,Ramirez,233
15,Barbara,Hurst,227


---
## Q3. Trouver l'auteur le plus apprécié
---

In [70]:
query = """
        SELECT users.id, users.first_name, users.last_name, COUNT(likes.id) AS total_likes
        FROM users
        JOIN posts ON users.id = posts.user_id
        JOIN likes ON posts.id = likes.post_id
        GROUP BY users.id
        ORDER BY total_likes DESC
        LIMIT 1
"""
df_top_author = pd.read_sql_query(query, connection, index_col='id')
df_top_author


,first_name,last_name,total_likes
id,,,
57,Teresa,Moore,647


---
## Q4. Qui sont les auteurs des 3 articles les plus likés ?
---

In [69]:
query = """
        SELECT posts.id, posts.title, users.id AS user_id, users.first_name, users.last_name, COUNT(likes.post_id) AS nombre_de_likes
        FROM posts
        JOIN users ON posts.user_id = users.id
        JOIN likes ON posts.id = likes.post_id
        GROUP BY posts.id, posts.title, users.id, users.first_name, users.last_name
        ORDER BY nombre_de_likes DESC
        LIMIT 3
"""
df = pd.read_sql_query(query, connection, index_col='id')
df


,title,user_id,first_name,last_name,nombre_de_likes
id,,,,,
143,Half imagine another.,72,Melissa,Henry,84
83,Side foot leader popular.,63,Cynthia,Raymond,82
99,Area paper whatever mean.,64,Alexander,Cook,81


---
## Q5. Combien de personnes ont aimé au moins une publication ?
---

In [19]:
query = "SELECT COUNT(DISTINCT user_id) FROM likes"
cursor.execute(query)
nombre_personnes = cursor.fetchone()[0]
print(f"Nombre de personnes ayant aimé au moins une publication : {nombre_personnes}")


Nombre de personnes ayant aimé au moins une publication : 49


---
## Q6. Calculez le nombre cumulé de likes par jour
---

In [58]:
query = """
        SELECT created_at, COUNT(*) AS likes_par_jour,
        SUM(COUNT(*)) OVER (ORDER BY created_at) AS cumul_likes
        FROM likes
        GROUP BY created_at
        ORDER BY created_at
"""
df_likes_cumul = pd.read_sql_query(query, connection)
df_likes_cumul


,created_at,likes_par_jour,cumul_likes
0,2019-01-01,24,24
1,2019-01-02,34,58
2,2019-01-03,40,98
3,2019-01-04,36,134
4,2019-01-05,27,161
...,...,...,...
331,2019-12-24,35,9882
332,2019-12-25,33,9915
333,2019-12-26,32,9947
334,2019-12-27,23,9970


---
## Q7. (Optional) Qui est le(s) plus grand(s) fan(s) de chaque auteur ?

<p>
Le ou les plus grands fans d'un auteur sont définis comme le ou les utilisateurs qui ont le plus aimé les publications de l'auteur. c'est-à-dire que s'il y a égalité entre les fans qui ont tous deux aimé un auteur 20 fois, les deux fans doivent être renvoyés avec leur nombre de likes et l'auteur en question.</p>

<p>Indice : Vous pouvez utiliser <code>WITH</code></p>

---

In [62]:
query = """
        WITH AuthorLikes AS (
            SELECT
                p.user_id AS author_id,
                l.user_id AS fan_id,
                COUNT(*) AS nombre_de_likes
            FROM
                posts p
            JOIN likes l ON p.id = l.post_id
            GROUP BY
                p.user_id, l.user_id
        ), MaxLikes AS (
            SELECT
                author_id,
                MAX(nombre_de_likes) AS max_likes
            FROM
                AuthorLikes
            GROUP BY
                author_id
        )
        SELECT
            al.author_id,
            al.fan_id,
            u.first_name || ' ' || u.last_name AS fan_name,
            al.nombre_de_likes
        FROM
            AuthorLikes al
        JOIN MaxLikes ml ON al.author_id = ml.author_id AND al.nombre_de_likes = ml.max_likes
        JOIN users u ON al.fan_id = u.id
        ORDER BY
            al.author_id, al.nombre_de_likes DESC
"""
df_biggest_fans = pd.read_sql_query(query, connection, index_col='author_id')
df_biggest_fans


,fan_id,fan_name,nombre_de_likes
author_id,,,
50,43,Michael Allen,12
51,29,Kaylee Ball,21
52,15,Barbara Hurst,16
52,31,Gary Garza,16
53,12,Donna Ramirez,3
53,20,Scott Thompson,3
53,34,Brandy Mason,3
54,20,Scott Thompson,8
54,37,Bruce Chen,8
